#### Testing combinations with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now).

##### Idea for calculation

Using the kernel structure used in test_combinations, we can  generate combinations arrays for every event. 

For that, each event is assigned a block ( i.e. each block in the grid serves as an event), and the combinations is calculated for each block using the `starts[block_index]` and event length.
This is appended to `left` and `right` arrays. 

In [1]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [2]:
# The generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)

In [3]:
cumul_arr = numpy.zeros(base_len+1)
cumul_arr[0] = 0
cumul_arr[1:] = numpy.cumsum(base_arr)

In [4]:
start = cumul_arr[:-2]
stop = cumul_arr[1:-1]

In [5]:
start.shape

(31,)

In [6]:
stop[:5]

array([  3.,   8.,  11.,  12.,  16.])

In [7]:
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*lengths)
lengths_arr = numpy.array([base_len-1]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)
start = start.astype(numpy.int32)

In [8]:
pairs_lengths.shape

(32,)

In [9]:
lengths.shape

(31,)

##### Error 

Invalid memory access for array start. Can't figure out why this is happening.

In [10]:
# Now let's form the cuda function

mod = SourceModule('''
__global__ void comb_events(int* left,int* right,int* start,int* length,int* lengths,int* pairs_lengths)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j =  blockIdx.y*blockDim.y+threadIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if (i <length[0])
    {
    int temp = start[i];
    if (j< lengths[i] && k<lengths[i])
        {
            left[pairs_lengths[i] + j*lengths[i] + k] = j + temp;
            right[pairs_lengths[i] + j*lengths[i] + k] = k + temp;
        }
    }
}
''')

In [11]:
func = mod.get_function("comb_events")

In [12]:
left = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)

In [13]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(lengths_arr),
    cuda.In(lengths), cuda.In(pairs_lengths), block=(1, 8, 8), grid=(base_len-1, 1))

In [14]:
left

array([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,
        4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
        8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 12, 12, 12, 12, 13, 13, 13,
       13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 17, 17, 17, 17, 18, 18, 18,
       18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 22, 23, 23, 23, 24, 24, 24,
       25, 25, 25, 26, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 32, 32,
       32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36,
       36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39,
       40, 40, 40, 40, 40, 41, 41, 42, 42, 43, 44, 45, 45, 45, 45, 46, 46,
       46, 46, 47, 47, 47, 47, 48, 48, 48, 48, 49, 50, 50, 50, 50, 50, 51,
       51, 51, 51, 51, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 54, 54, 54,
       54, 54, 55, 55, 55, 56, 56, 56, 57, 57, 57, 58, 58, 58, 59, 59, 59,
       60, 60, 60, 61, 62, 62, 63, 63, 64, 65, 66, 66, 66, 67, 67, 67, 68,
       68, 68])

In [20]:
#Let's reshape it
numpy.split(left, pairs_lengths[1:].astype(numpy.int))

[array([0, 0, 0, 1, 1, 1, 2, 2, 2]),
 array([3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7,
        7, 7]),
 array([ 8,  8,  8,  9,  9,  9, 10, 10, 10]),
 array([11]),
 array([12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15]),
 array([16]),
 array([17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20]),
 array([21]),
 array([], dtype=int32),
 array([22]),
 array([23, 23, 23, 24, 24, 24, 25, 25, 25]),
 array([26]),
 array([27]),
 array([28, 28, 28, 29, 29, 29, 30, 30, 30]),
 array([31]),
 array([32, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35]),
 array([36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 39, 39,
        39, 39, 39, 40, 40, 40, 40, 40]),
 array([41, 41, 42, 42]),
 array([43]),
 array([44]),
 array([45, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 47, 48, 48, 48, 48]),
 array([49]),
 array([50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 52, 52, 52, 52, 52, 53, 53,
        53, 53, 53, 54, 54, 54, 54, 54]),
 array([5